In [1]:
%%capture
%pip install specklepy

# dotenv is a library that allows you to load environment variables from a .env file
%pip install python-dotenv
%reload_ext dotenv
%dotenv

In [2]:
# boilerplate user credentials and server

import os
HOST_SERVER = os.getenv('HOST_SERVER')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')

In [3]:
from specklepy.api.client import SpeckleClient

client = SpeckleClient(host=HOST_SERVER)  # or whatever your host is
client.authenticate_with_token(ACCESS_TOKEN)  # or whatever your token is

In [4]:
stream_id = "01f76c41dc"  # or whatever your stream id is

from specklepy.transports.server import ServerTransport
from specklepy.api.wrapper import StreamWrapper

transport = ServerTransport(client=client, stream_id=stream_id)

stream = client.stream.get(stream_id)

In [10]:
commits = client.commit.list(stream_id=stream_id, limit=2)

from specklepy.api import operations

# get obj id from latest commit
latest = commits[0].referencedObject

# receive objects from speckle
latest_data = operations.receive(obj_id=latest, remote_transport=transport)

latest_data["new_data"] = "hello world"




'hello world'

In [15]:
# send objects to speckle

# create a new commit
obj_id = operations.send(base=latest_data, transports=[transport])

commit_id = client.commit.create(
    stream_id=transport.stream_id,
    object_id=obj_id,
    message="New Data"
)

In [16]:
commits = client.commit.list(stream_id=stream_id, limit=2)

from specklepy.api import operations

# get obj id from latest commit
now = commits[0].referencedObject

# receive objects from speckle
now_data = operations.receive(obj_id=latest, remote_transport=transport)